# Customer Churn Prediction (ANN)

## Description

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Importing Required Libraries

In [11]:
import numpy as np
import pandas as pd

In [13]:
import tensorflow as tf

#### Checking if TensorFlow is using GPU properly

In [19]:
gpus = tf.config.list_physical_devices('GPU') #this is the main code for listing all available GPU(s)
if gpus:
    print(f"✅ TensorFlow detected {len(gpus)} GPU(s):")
    for gpu in gpus:
        print(f"  - {gpu}")
else:
    print("❌ No GPU detected. TensorFlow is using CPU instead.")

✅ TensorFlow detected 1 GPU(s):
  - PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


<hr style="height:1px;border-width:0;color:black;background-color:black">

### Section 1: Data Preprocessing

In [166]:
df = pd.read_csv('C:\\Users\\Amin\\DeepLearningProjects\\my-codes\\Data\\Churn_Modelling.csv')

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [30]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


It seems that the first three features (columns) are not useful and as a resutl, we will drop those features. Plus, we will split our data into features (X) and targets (y)

In [147]:
X = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
y = df['Exited']

In [149]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [151]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

#### Another Approach

In [153]:
X = df.iloc[:, 3:-1]
y = df.iloc[:, -1]

In [155]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [137]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [139]:
X = X.values
y = y.values

In [141]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [143]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Section 2: Encoding Categorical Data

Due to the fact that the input of Deep Learning must be numerical, we have to encode the categorical features (like Geography and Gender in this dataset)

#### One-Hot Encoding to the 'Gender' column (using pd.get_dummies())

In [168]:
X = pd.get_dummies(X, columns=['Gender'], drop_first=True)

In [176]:
X['Gender_Male'] = X['Gender_Male'].astype(int)

In [178]:
X.head()

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male
0,619,France,42,2,0.00,1,1,1,101348.88,0
1,608,Spain,41,1,83807.86,1,0,1,112542.58,0
2,502,France,42,8,159660.80,3,1,0,113931.57,0
3,699,France,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,43,2,125510.82,1,1,1,79084.10,0


---

#### One Hot Encoding for 'Geoghraphy' Column (using library)

In [68]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

This approach should be implemented on numpy array because if it applies to the pandas dataframe, the display of numbers will be different.

In [188]:
ct.fit_transform(X)[0]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 6.1900000e+02,
       4.2000000e+01, 2.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0134888e+05, 0.0000000e+00])

In [108]:
X[0]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

In [199]:
X = X.values
y = y.values

In [201]:
# Apply OneHotEncoder to 'Geography' (column index 1)
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')

In [203]:
X = ct.fit_transform(X)

In [205]:
X[0]

array([1.0, 0.0, 0.0, 619, 42, 2, 0.0, 1, 1, 1, 101348.88, 0],
      dtype=object)

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Section 3: Scaling the Features

Deep Learning models require scaled data. Therefore, we need to scale them before training our model

In [213]:
from sklearn.preprocessing import StandardScaler

In [215]:
scaler = StandardScaler()

In [217]:
X[0]

array([1.0, 0.0, 0.0, 619, 42, 2, 0.0, 1, 1, 1, 101348.88, 0],
      dtype=object)

In [225]:
X = scaler.fit_transform(X)

In [227]:
X[0]

array([ 0.99720391, -0.57873591, -0.57380915, -0.32622142,  0.29351742,
       -1.04175968, -1.22584767, -0.91158349,  0.64609167,  0.97024255,
        0.02188649, -1.09598752])

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Section 4 : Splitting the Data

#### Splitting the Data set into Training set and Test set

In [238]:
from sklearn.model_selection import train_test_split

In [240]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [244]:
len(X_train)

6700

In [246]:
len(X_test)

3300

In [248]:
len(y_train)

6700

In [250]:
len(y_test)

3300

<hr style="height:1px;border-width:0;color:black;background-color:black">

### Section 5: Bulding the ANN